# Prototype notebook : Querying data across all Omix Atlas
_For internal use only_

### Instructions:

1. Please run all code cells one by one.
2. User is required to enter an SQL query to search for the required datasets.

## Install polly-python

In [ ]:
!sudo pip3 install polly-python joblib

In [11]:
from polly.omixatlas import OmixAtlas
from polly.auth import Polly
from joblib import Parallel, delayed
import pandas as pd
import time
import re

ERROR! Session/line number was not unique in database. History logging moved to new session 134


## Authentication with Polly

In [12]:
AUTH_TOKEN = "eyJjdHkiOiJKV1QiLCJlbmMiOiJBMjU2R0NNIiwiYWxnIjoiUlNBLU9BRVAifQ.GrUC4WVU8h6EEo-uUTaI1ESZ0Wx6lBa4kH_xeOyY6Hhg_EFd3aVx7sD6x7rQX21UMqRjvgbITIRXB7ops7ZqkuiWwFZNE9bdHsFhxR67RxtPm4lQImgT7awTpKr4nVt8rY57SNV7ik-Cj_bTg_VSZ6IrFfBZWHCUQlwtEoXRHPPFg6Q51Pn06DRBAUi7P01X28AXD6NRMf4uvUpnx9KmeEHMRCrXHaS2pBA-y0JsWlJKjrM2M5qTbjDC1BKhp9xddIzZ2GD5_XXk4jofSYStKcG8O-QV9m6GQX4Ho0_u3DN9ukWWO8ujOjs1Wn8ZnfhfSsabgaFx21QfRSYW8iU5nw.AFgOMSew5txA6gF0.Kos8yANc2FrZhzqRoqk-SwdKKgsBQZtXKpTE6KwFI-vbAc7rWs04qjaTD1Kqcg7un5nOR_TXpjdn3at0G_dBpPs8Hp1o4Z8LHna847__aQiu06Nmcvld40idJDPxNCouJGRICVZLzTSKt9dtdp0DZnEO5Y1bKWlG7A-3kr8e4Bfa4kxHB5DhujXImYuttQv-Fb8oMFknkn3h1CWaf0rKmZ3fRsKnPUW-TmKc0JEk1A6d0_xIZssBjnpL4Tbd5pdg6ZkdGoy7GDzArE9JEFZBG_BIMiJHT2tkJc_Y67iWl2vWdSZ5OLNpGojgv3j1t-2fuHKcTLo0TFbXBvzAI_eKzy9r4wB5aEH_4k34FPlqh3M8kpYRq0DzmxXFHXLKvS1gfpQbHub5dziwIIOspitBuC4ps963dnE-1WledZTIsfzIiOESW5nKEccwPZ0OlAmcOiCkx67gwV5DIdcJoc9ESZnzLB8c7kqf0cbSMlBp4K36QADe2UwGBt7Nc54a_XsBQ2cpdCbKxK342NbZzqemZNuwBLwATp6-9yWo13gRDMlIJg1dKRwEB5cFlVsPVmQc0jrB1hohjX4rlJiAz5vBFpe8IYVwUKw9yFRh_dw_1ugKZAiMqCAadKZjonAc4QpwMSZLEKpj-r9Sxtm9h__ohtpMxl3QwtRofsBSzVBODsQV16M7cqlgJ8jRhv2Fo-nzT62Glv64FUN-jNmv9b7iEoNWM8mt9VV2hjt2fGcTWlZwXH_sSPQ2P70JxhSWGAK8yTCg6CjLhViqq_Dz49mrVS4cdfKVsv3H6agDUzl8dO0B01UFOUR0nuwbKscyjK8ieJ4VGK4trFy43HNBF9MNuGNdUte6ppWVlcbf5nRplq9RMMnrVeSmiRTmw-o6lnhARJuL3TOuWbb99zE_a84LcqlWtK2xDXIey3xV5b3VcJdsf1abu3MAUnsN2OCixZCWMp9iJikQsu0dn6HedUPxf4VvxTKkjAD6qLZT6pNgsaYzAvhilrcg9qrPa1yXQoXeT0Y1jhl1xzOcg0ljp-nk149pRnAbP7aLH08TArxcRW--e_Eb2SxGYgZuoR8GE6KQf4GQdJl92wX13Ynz6oQ3eNGj8o6vcz-3a3kdXsN92_XI3-1sqC5HbOvUE5q6DmEnWfJwsxiySn6VmgmRa48BJwrDNXM4aaFIZhTxM08jvOKUU82rQqTyQAW4134P53C8PrhUf9hGO0aLVT2ZuHBit541JEiHSu21GKmbJaE8Slsr4H-riNnDDiXD8c0YscBt7MMIhfeWY8rxw49wBn3FwbgTZdlGgdIyPFQTCXcKBD6kGa7RE1-YFb-V4Zs.hMASyWDl6LEVzoB5aaAsjQ"
Polly.auth(AUTH_TOKEN)

omixatlas = OmixAtlas()

## Searching across all OA

In [ ]:
def query_split(query):
    
    """
    Returns N SQL queries equal to the number of input indexes
    
    Extended description of function.
    
    Parameters:
    query (str): input SQL query to search across any given omixatlases
    
    Returns:
    q_list (list): list of SQL queries where each query queries on a single index only.
    
    """
    q_list = []
    
    query = query.replace("\n", "") #remove new line from query
    
    if('union' in query.lower()):
        q_list = re.split('union', query, flags=re.IGNORECASE)
        q_list = [q.strip() for q in q_list]
    else:
        index_search = re.findall(r"from.*\.[a-z]*s", query, re.IGNORECASE)[0].split(',')
        index_list = [re.sub("from ", "", index) for index in index_search]
        index_list = [index.strip() for index in index_list]

        const_string1 = query.split(index_list[0]) #Extract constant SQL query
        const_string2 = query.split(index_list[-1]) #Extract constant SQL query
    
        for index in index_list:
            q = f"{const_string1[0]}{index}{const_string2[-1]}"
            q_list.append(q)
    
    return q_list

def query_oa(query, version):
    
    """
    Returns a table containing metadata for all results based on the input SQL query
    
    Extended description of function.
    
    Parameters: 
    query (string): list of input SQL queries where each query is used to query across a single omixatlas
    version (string): The API version of data infra on which to query
    
    Returns:
    all_df (dataframe): DataFrame containing metadata of the resulting data.
    
    """

    all_df = omixatlas.query_metadata(query, query_api_version=version)
    return all_df
    

def parallel_query(query_list, version):
    
    """
    Performs a parallel execution by dividing the task among multiple threads rather than multiple CPUs
    
    Extended description of function.
    
    Parameters: 
    query_list (list): list of input SQL queries where each query is used to search across a single omixatlas
    version (string): The API version of data infra on which to query
    
    Returns:
    df (dataframe): DataFrame containing dataset level metadata of the resulting datasets.
    
    """

    final = []
    result = Parallel(n_jobs=4, prefer="threads")(delayed(query_oa)(query, version) for query in query_list)

    for that_dict in result:
        if isinstance(that_dict, pd.DataFrame):
            final.append(that_dict)
    
    df = pd.concat(final)
    return df

def empty_df():

    """
    Creates an empty dataframe with a message when no data is returned
    
    Extended description of function.
    
    Returns:
    empty_df (dataframe): An empty dataframe with a message.
    
    """

    empty_df = pd.DataFrame()
    data = pd.DataFrame({"Message": "No Data to show!"}, index=[0])
    empty_df = empty_df.append(data)
    return empty_df

def query_all_oa(sql, api_ver):

    """
    Entry point for the cross omixatlas querying app.
    
    Extended description of function.
    
    Parameters: 
    sql (string): Input SQL query
    api_ver (string): The API version of data infra on which to query
    
    Returns:
    result_df (dataframe): DataFrame containing dataset level metadata of the resulting datasets.
    status (string): String containing messages for the user
    time_elapsed (string): Execution wall-time in seconds
    """
    
    start_time = time.time()
    result_df = pd.DataFrame()

    try:

        result_df = query_oa(sql, api_ver)
        if result_df.empty:
            status = 'Query successful, No datasets were returned'
        else:
            status = 'Query Successful'
    
    except Exception as e:

        if 'Different mappings' in repr(e) or 'SYNTAX_ERROR' in repr(e):
            query_list = query_split(sql)
            result_df = parallel_query(query_list, api_ver)
            if result_df.empty:
                status = 'Query successful, No datasets were returned'
                result_df = empty_df()
            else:
                status = 'Query Successful, but caught an Exception with message: One or more indexes in SQL query cannot be concatenated due to reasons unknown. The RCA is yet to be found'
        else:
            status = repr(e)
            result_df = empty_df()
    
    time_elapsed = str("Elapsed time = --- %s seconds ---" % (time.time() - start_time))
    return(result_df, status, time_elapsed)

# Input 

In [ ]:
input_query = f"""SELECT * from geo.datasets, pcd.datasets WHERE disease = 'Carcinoma, Hepatocellular' LIMIT 2000"""
version = 'v1'

res, stat, wall_time = query_all_oa(input_query, version)